## Used Tractor Price Estimator - Model Training

In this notebook, data that was collected from Farmers Weekly (https://classified.fwi.co.uk/used/a-Tractors/24/b-Tractors/95/) is used to train a model to estimate the prices of John Deere tractors for sale on the used tractor market. Minimal data cleaning is required as the data was put together with prediction-making in mind. More data is currently being collected in order to enhance model performance and increase the overall accuracy of the model.



In [1]:
# Import packages
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error

In [2]:
# Import dataset and view first 5 lines
df = pd.read_csv("johndeere_tractordata.csv")
df.head()

,Make,Model,Hours,Year,HP,Weight in kg,Price ex VAT,ID,Front Loader
0,John Deere,6250R,2137,2019,250,9300,128750,100,no
1,John Deere,6125R,3330,2014,125,5910,46000,101,no
2,John Deere,6110MC,679,2016,116,4700,41995,102,yes
3,John Deere,5090R,5216,2011,90,3700,23750,103,no
4,John Deere,6155R Ultimate,2825,2018,155,7100,82000,104,no


In [3]:
# Make sure our datatypes are correct
df.dtypes

Make            object
Model           object
Hours            int64
Year             int64
HP               int64
Weight in kg     int64
Price ex VAT     int64
ID               int64
Front Loader    object
dtype: object

Instead of using the year value as an int for making predictions, we will put each instance into a bin depending on the production year of the tractor. For example, a tractor produced in 2003 will go into a bin containing all tractors produced between 2000 and 2004. This separates newer tractors from older ones in a way that will make it easier for the model to differentiate between different ages of tractor compared to keeping all of the figures as integers.

In [4]:
# Divide years into bins: 5 different ranges and replace the original dataframe values with the new ones
bins= [0,2000,2005,2010,2015,2021]
labels = ['-2000','2000-2004','2005-2009','2010-2014','2015-present']
df['Year'] = pd.cut(df['Year'], bins=bins, labels=labels, right=False)
df.head()

,Make,Model,Hours,Year,HP,Weight in kg,Price ex VAT,ID,Front Loader
0,John Deere,6250R,2137,2015-present,250,9300,128750,100,no
1,John Deere,6125R,3330,2010-2014,125,5910,46000,101,no
2,John Deere,6110MC,679,2015-present,116,4700,41995,102,yes
3,John Deere,5090R,5216,2010-2014,90,3700,23750,103,no
4,John Deere,6155R Ultimate,2825,2015-present,155,7100,82000,104,no


The feature "Front Loader" tells us whether a tractor has a front loader or not. A front loader will add a few thousand to the price of any tractor; if we had two tractors with the exact same specifications, and one has a front loader while the other doesn't, the one with the front loader should be priced higher than the one without a front loader. 

This is a categorical feature so we need to do one-hot encoding to create dummies for this feature. We will also do the same for the year ranges we produced in the previous cell. These are placed in a temporary dataframe that we will concatenate with the original dataframe.

In [5]:
# Create dummies for the "Front Loader" feature and place them in a temporary dataframe
features = ["Front Loader", "Year"]
temp_df = pd.get_dummies(df[features])
temp_df.head()

,Front Loader_no,Front Loader_yes,Year_-2000,Year_2000-2004,Year_2005-2009,Year_2010-2014,Year_2015-present
0,1,0,0,0,0,0,1
1,1,0,0,0,0,1,0
2,0,1,0,0,0,0,1
3,1,0,0,0,0,1,0
4,1,0,0,0,0,0,1


In [6]:
# Add dummy features to original dataframe
df = pd.concat([df, temp_df], axis=1)
df.head()

,Make,Model,Hours,Year,HP,Weight in kg,Price ex VAT,ID,Front Loader,Front Loader_no,Front Loader_yes,Year_-2000,Year_2000-2004,Year_2005-2009,Year_2010-2014,Year_2015-present
0,John Deere,6250R,2137,2015-present,250,9300,128750,100,no,1,0,0,0,0,0,1
1,John Deere,6125R,3330,2010-2014,125,5910,46000,101,no,1,0,0,0,0,1,0
2,John Deere,6110MC,679,2015-present,116,4700,41995,102,yes,0,1,0,0,0,0,1
3,John Deere,5090R,5216,2010-2014,90,3700,23750,103,no,1,0,0,0,0,1,0
4,John Deere,6155R Ultimate,2825,2015-present,155,7100,82000,104,no,1,0,0,0,0,0,1


We can now remove the features we created dummies for, as we no longer need them for our model. There are also several other features we can remove at this point, after which we will begin training our model.

In [7]:
# Drop features we no longer need
df = df.drop(columns=["Front Loader", "Year", "Make", "Model", "ID"])
df.head()

,Hours,HP,Weight in kg,Price ex VAT,Front Loader_no,Front Loader_yes,Year_-2000,Year_2000-2004,Year_2005-2009,Year_2010-2014,Year_2015-present
0,2137,250,9300,128750,1,0,0,0,0,0,1
1,3330,125,5910,46000,1,0,0,0,0,1,0
2,679,116,4700,41995,0,1,0,0,0,0,1
3,5216,90,3700,23750,1,0,0,0,0,1,0
4,2825,155,7100,82000,1,0,0,0,0,0,1


We train our model using a 70/30 train/test split. A cross validation technique may be used in future once more data has been collected for training.

In [8]:
# Isolate the descriptive feature from the training set
X = df.drop(columns=["Price ex VAT"])
Y = np.array(df['Price ex VAT'])

# Split the data into training and test sets
X_train, X_test, y_train, y_test \
    = train_test_split(X, Y, \
                        shuffle=True, \
                        train_size = 0.7)

In [9]:
# Define and train a Random Forest Regressor model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Make predictions based on the test set and save the accuracy score
y_pred=model.predict(X_test)

# Calculate the absolute errors
errors = abs(y_pred - y_test)
errors

array([12421.03,  3926.16, 10189.94, 16972.92, 70206.79, 21758.32,
       10214.44,  2819.62,  4600.06,  6588.76,  2439.54])

In [10]:
# Print the Mean Absolute Error
print('Mean Absolute Error:', round(np.mean(errors), 2))

Mean Absolute Error: 14739.78


In [11]:
# Calculate and print the figures for Root Mean Squared Error and R-squared
test_set_rmse_1 = (np.sqrt(mean_squared_error(y_test, y_pred)))
test_set_r2_1 = r2_score(y_test, y_pred)
print("RMSE: ",test_set_rmse_1)
print("R2: ",test_set_r2_1)

RMSE:  23639.024298573055
R2:  0.6054568045729344


As we add more data to the dataset (it currently consists of 33 rows), we will get to a stage where we can go deeper into feature engineering and algorithm tuning in order to increase model performance.